# Building a Spam Filter with Naive Bayes
In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to write a program that detects and classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

The data consist of 5,572 SMS messages that already classified by humans to spam or ham (non-spam).

## Exploring the Dataset
We'll now start by reading in the dataset.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 500)
data = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
print(data.shape)
data.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [2]:
data['Label'].value_counts(normalize=True)* 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

Above we can see that approximately 87 % of the messages are `ham` non spam, while 13 % are spam. This sample looks representative, as in realty most of messages people receive are non spam.

## Data Cleaning

We will begin with cleaning the data to bring to a form that we can work with and perform the Naive Bayes algorithm to achieve our goal of classifying the SMS messages accurately.

In [3]:
# use regex to remove any charachter except word charachter, then lower all the words 
data['SMS'] = data['SMS'].str.replace('\W', ' ').str.lower()
data['SMS']

0                                                        go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   
1                                                                                                                                          ok lar    joking wif u oni   
2            free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005  text fa to 87121 to receive entry question std txt rate t c s apply 08452810075over18 s
3                                                                                                                      u dun say so early hor    u c already then say   
4                                                                                                          nah i don t think he goes to usf  he lives around here though
                                                                                      ...                                                                  

## Spliting the Training and Test Set

Next we are going to split our dataset into a training and a test set, the training set will be 80% of the data,while the test set will be the remaining 20%. Also we are going randomize the entire dataset to ensure that `spam` and `ham` messages are spread properly throughout the dataset.

In [4]:
randomized_data = data.sample(frac=1, random_state=1)
train_test_length = round(len(randomized_data)*0.8)
train_set = randomized_data[:train_test_length].reset_index(drop=True)
test_set = randomized_data[train_test_length:].reset_index(drop=True)

# make sure we maintained the same percentage between ham and spam
print('1- Percentage of Spam and Non-Spam in Train Set', '\n')
print(train_set['Label'].value_counts(normalize=True)*100, '\n')
print('2- Percentage of Spam and Non-Spam in Test Set', '\n')
print(test_set['Label'].value_counts(normalize=True)*100)

1- Percentage of Spam and Non-Spam in Train Set 

ham     86.54105
spam    13.45895
Name: Label, dtype: float64 

2- Percentage of Spam and Non-Spam in Test Set 

ham     86.804309
spam    13.195691
Name: Label, dtype: float64


# Constructing the data

Now we will start to organize the data in a shape that compatible the algorithm. But before that we can give small brief about the algorithm and the math behind it.

The Naive Bayes Algorithm help the computer to learn how we classify messages (what counts as spam and non-spam for us), and then it uses the human knowledge (messages classified beforehand by humans) to estimate probabilities for new messages.

The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

$$
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
$$$$
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
$$

Also, to calculate $ P(w_i|Spam) $ and $ P(w_i|Ham) $ inside the formulas above, we'll need to use these equations:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$

$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

Let's now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [5]:
# first split the words in the messages, then iterating over the splitted words and append them to a list 
train_set['SMS'] = train_set['SMS'].str.split()
vocabulary = []
for sms in train_set['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = (list(set(vocabulary)))

## The Final Training Set

We're now going to use the vocabulary we just created to make the data transformation we want.

In [6]:
# initiate a dictionary that contain the occurrence of each word per index 
word_counts_per_sms = {unique_word: [0] * len(train_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(train_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [7]:
# transform the dictionary to pandas data frame
word_count = pd.DataFrame(word_counts_per_sms)
word_count.head()

,canceled,dictionary,cha,hopeing,re,website,brdget,project,pract,darlin,...,bong,shame,www,concentrate,gandhipuram,roles,hmv1,creepy,w8in,mandy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# concat the word count data frame with the original train set data frame
training_set = pd.concat([train_set, word_count], axis=1)
training_set.head()

,Label,SMS,canceled,dictionary,cha,hopeing,re,website,brdget,project,...,bong,shame,www,concentrate,gandhipuram,roles,hmv1,creepy,w8in,mandy
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me, moan]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a, card, on, da, present, lei, how, ü, all, want, 2, write, smth, or, sign, on, it]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now lets calculate the constants in the four equations mentioned earlier considering that these constants will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

 - $P(Spam)$ 
 - $P(Ham)$
 - $N_{Spam}$ 
 - $N_{Ham}$ 
 - $N_{Vocabulary}$

We'll also use Laplace smoothing and set $\alpha = 1$.

In [9]:
# calculate P Spam
p_spam = len(training_set[training_set['Label'] == 'spam'])/len(training_set)
p_spam

0.13458950201884254

In [10]:
# calculate P Ham
p_ham = len(training_set[training_set['Label'] == 'ham'])/len(training_set)
p_ham

0.8654104979811574

In [11]:
# calculate N Spam
spam_msgs = training_set[training_set['Label'] == 'spam']
n_spam = spam_msgs['SMS'].apply(len).sum()
n_spam

15190

In [12]:
# calculate N Ham
ham_msgs = training_set[training_set['Label'] == 'ham']
n_ham = ham_msgs['SMS'].apply(len).sum()
n_ham

57237

In [13]:
# calculate N Vocabulary
n_vocabulary = len(vocabulary)
alpha = 1
print(n_vocabulary)

7783


## Calculating Parameters

Now we will calculate the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

In [14]:
# Initiate two dictionaries  for the parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

for word in vocabulary:
    no_words_given_spam = spam_msgs[word].sum()
    p_word_given_spam = (no_words_given_spam + alpha) / (n_spam + alpha * n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    no_words_given_ham = ham_msgs[word].sum()
    p_word_given_ham = (no_words_given_ham + alpha) / (n_ham + alpha * n_vocabulary)
    parameters_ham[word] = p_word_given_ham
    

## Classifying A New Message

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

Takes in as input a new message (w1, w2, ..., wn).
Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [15]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [16]:
# test the spam filter with a spam message
message_1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
classify(message_1)

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [17]:
# test the spam filter with a ham (non-spam) message
message_2 = 'Sounds good, Tom, then see u there'
classify(message_2)

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Spam Filter Accuracy

Finaly lets measure the spam filter accuracy using the `test_set`. We will modify the above to return the classification instead of printing it.

In [18]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [19]:
# use the function above to create new column contain the predicted classification
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,later i guess i needa do mcat study too,ham
1,ham,but i haf enuff space got like 4 mb,ham
2,spam,had your mobile 10 mths update to latest orange camera video phones for free save s with free texts weekend calls text yes for a callback orno to opt out,spam
3,ham,all sounds good fingers makes it difficult to type,ham
4,ham,all done all handed in don t know if mega shop in asda counts as celebration but thats what i m doing,ham


In [20]:
# initiate a counter to count the predicted correct classification 
correct = 0
total = len(test_set)
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', round((correct/total)*100, 2), '%')

Correct: 1100
Incorrect: 14
Accuracy: 98.74 %


The accuracy measured here it's exceeding the target accuracy set at the begining of this project. 